# Import packages

In [1]:
import math, os, collections
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from evaluation import Evaluation
from db_connection.utils import get_conn, get_data_start_dt
from utils_load import load_w103, load_w106, load_cust, cust_process
from mlaas_tools.config_build import config_set

In [2]:
## Configure env
if not os.path.isfile('config.ini'):
    config_set()

In [3]:
## Load db connection
rawdata_conn = get_conn('edu')

read key file
login as edu-cytsao


In [120]:
rawdata_conn = get_conn('edu')
# interaction train w103
w103_df = load_w103('2019-01-31', rawdata_conn, span=18)
purchase_hist_dca = pd.DataFrame(w103_df[w103_df['deduct_cnt']!=0].groupby("cust_no")["wm_prod_code"].apply(lambda x: list(set(x.values.tolist()))))
print(purchase_hist_dca.head())

read key file
login as edu-cytsao
                                wm_prod_code
cust_no                                     
++1QLn8bCFzxWN8vQYBuRg==              [NN18]
++2WLG2lyrgp9jhLkCFEKg==  [WW06, 2509, AG74]
++3bvnatF8Ie6ZC/bs3rcA==  [1204, 6017, 6216]
++7FzPdZNv2Gw99EJ/ZUUg==              [CC03]
++GMXRROXNCZtHYs3ymfyg==  [AG48, AG24, AG30]


In [121]:
rawdata_conn = get_conn('edu')
purchase_hist = w103_df.groupby("cust_no")["wm_prod_code"].apply(lambda x: list(set(x.values.tolist()))).to_dict()
evaluation = Evaluation('2019-01-31', None, '1m', purchase_hist)
evaluate_w103 = evaluation.read('2019-01-31', rawdata_conn, '1m')
evaluate_w103

read key file
login as edu-cytsao
read key file
login as edu-cytsao


,cust_no,wm_prod_code
0,00FmcV87beY8s7OI0Xjtkg==,5704
1,00IrxuhcWjiKQPXJDBv2Yg==,AG80
2,00IrxuhcWjiKQPXJDBv2Yg==,AG26
3,00IrxuhcWjiKQPXJDBv2Yg==,AC18
4,00roz6dL8l5Ouk54dlOc9A==,FC85
...,...,...
5664,zzjZadLsQ1PRT0/+2zULnA==,UF57
5665,zzOnaWQ8v9wgoMPXdIC63A==,BB01
5666,zzOnaWQ8v9wgoMPXdIC63A==,3706
5667,zzOnaWQ8v9wgoMPXdIC63A==,CCB7


In [122]:
evaluate_w103_hist = pd.DataFrame(evaluate_w103.groupby("cust_no")["wm_prod_code"].apply(lambda x: list(set(x.values.tolist()))))
evaluate_w103_hist

,wm_prod_code
cust_no,
+0pDrcJE0fvDekr6JLFYVQ==,[EE29]
+1Oc52w6Z5QfJwL+8BQ0Ew==,"[ED64, 8915, FE41]"
+1y5TbuipnX4nqzoKTbQCw==,[AG80]
+2nQW/qhjI4QlLqIh+3GIg==,[1338]
+3725e06hHfZnvl65bmRSg==,[UF58]
...,...
zy3vgn+M6LkwhWL0o3x82A==,[AF51]
zyOOIBQoa3exIwHf7YKzIA==,[AV03]
zyPzzDNfmL3JASvX2a5Thg==,"[UF51, FE41]"


In [123]:
train_eva_hist = purchase_hist_dca.merge(evaluate_w103_hist, how='inner', left_index=True, right_index=True)
train_eva_hist

,wm_prod_code_x,wm_prod_code_y
cust_no,,
+0pDrcJE0fvDekr6JLFYVQ==,"[XX05, AA91, CC74, PP56, NN03, CCG3, MM17, AG2...",[EE29]
+1y5TbuipnX4nqzoKTbQCw==,"[CC88, AA55, AG69, AG21, AV03, AG25]",[AG80]
+3725e06hHfZnvl65bmRSg==,[BB51],[UF58]
+984UCbOEorzWUOaGOaBfQ==,"[KK32, AA39]",[UF34]
+9W7hR/cahy4WlpB7UOgmw==,"[2018, AA77]",[AA05]
...,...,...
zuiSmmDqX+FNbpl+n5YmLg==,"[SS09, KK32, AW01]",[CCE2]
zy3vgn+M6LkwhWL0o3x82A==,"[KK32, JJ05]",[AF51]
zyOOIBQoa3exIwHf7YKzIA==,[AH11],[AV03]


In [124]:
train_eva_hist.loc[train_eva_hist['wm_prod_code_y'].isnull() == False]

,wm_prod_code_x,wm_prod_code_y
cust_no,,
+0pDrcJE0fvDekr6JLFYVQ==,"[XX05, AA91, CC74, PP56, NN03, CCG3, MM17, AG2...",[EE29]
+1y5TbuipnX4nqzoKTbQCw==,"[CC88, AA55, AG69, AG21, AV03, AG25]",[AG80]
+3725e06hHfZnvl65bmRSg==,[BB51],[UF58]
+984UCbOEorzWUOaGOaBfQ==,"[KK32, AA39]",[UF34]
+9W7hR/cahy4WlpB7UOgmw==,"[2018, AA77]",[AA05]
...,...,...
zuiSmmDqX+FNbpl+n5YmLg==,"[SS09, KK32, AW01]",[CCE2]
zy3vgn+M6LkwhWL0o3x82A==,"[KK32, JJ05]",[AF51]
zyOOIBQoa3exIwHf7YKzIA==,[AH11],[AV03]


In [125]:
train_eva_hist = train_eva_hist.dropna()
dca_buy_again = []
for i in range(len(train_eva_hist)):
    if type(train_eva_hist.iloc[i]['wm_prod_code_x']) != list or type(train_eva_hist.iloc[i]['wm_prod_code_y']) != list:
        dca_buy_again += ['NA']
    elif len(set(train_eva_hist.iloc[i]['wm_prod_code_x']) & set(train_eva_hist.iloc[i]['wm_prod_code_y'])) > 0:
        dca_buy_again += ['y']
    else:
        dca_buy_again += ['n']
train_eva_hist['dca_buy_again'] = dca_buy_again

In [126]:
train_eva_hist[train_eva_hist['dca_buy_again']=='y']

,wm_prod_code_x,wm_prod_code_y,dca_buy_again
cust_no,,,
+ABoMPDclH9sdmFKkvHTwQ==,[KK41],[KK41],y
+LcBXj1EFcZfmE2fFiSOeA==,[CC79],[CC79],y
+VfDtNP136dC9fJ/HjpR0Q==,"[II33, BB51, NN83, CC87, NN68, NN23]",[CC87],y
+wDHmG+i5jRbbBj1yZ9xAw==,"[NN83, QQ49, NN84, 2143]",[NN83],y
16r0LJheBSV50oGJIW2ZFw==,"[QQ31, AG24, 2128, CC55, AAD9, CC88, 2124, 271...",[AG23],y
...,...,...,...
y4gVUcMUoGm08YOWVSDYwA==,"[BB16, PP65, KK08, OB08]",[KK08],y
yYu0lAMplnq+QudtpD91rQ==,"[BB39, AGE4]",[BB39],y
z0OVVSdUUHBB2O3FsmvGwQ==,"[NN82, BB51, KK09, KK68, BB39, KK41]",[BB39],y


In [119]:
'''
evaluate total/evaluate dca(在training時買過定期定額)/duplicate(又買之前申購過dca的基金)
2018-12-31 18m 3883/1454/203
2019-01-31 18m4101/1544/184
'''

'\nevaluate total/evaluate dca/duplicate\n2018-12-31 18m 3883/1454/203\n'

In [ ]:
# conclusion: 存在加碼申購定期定額的user，無法保證推薦中拿掉已經申購過的基金會提升成績